In [13]:
import pymysql
import pandas as pd
from IPython.display import display, clear_output
from ipywidgets import Dropdown
import glob
import os
import openpyxl
connect = pymysql.connect(host='127.0.0.1', user='root', password='0000', db='practice', charset='utf8')
cur = connect.cursor()

test = "SELECT * FROM practice.m_area;"
test1 = "SELECT * FROM practice.m_store;"
query1 = "SELECT * FROM practice.tbl_order_202105;"

cur.execute(test)
test = pd.DataFrame(cur.fetchall())
test.columns = ['area_cd', 'wide_area', 'narrow_area']

cur.execute(test1)
test1 = pd.DataFrame(cur.fetchall())
test1.columns = ['store_id', 'store_name', 'area_cd']


cur.execute(query1)
data1 = pd.DataFrame(cur.fetchall())
data1.columns = ['order_id', 'store_id', 'customer_id', 'coupon_cd', 'sales_detail_id',
                 'order_accept_date', 'delivered_date', 'takeout_flag', 'total_amount', 'status']

current_dir = os.getcwd()
tbl_order_file = os.path.join(current_dir,'tbl_order_*.csv')
tbl_order_files = glob.glob(tbl_order_file)



order_all = pd.DataFrame()
for file in tbl_order_files:
    order_tmp = pd.read_csv(file)
    print(f'{file}:{len(order_tmp)}')
    order_all = pd.concat([data1,order_tmp],ignore_index=True)

order_all = data1.loc[data1['store_id']!=999]

order_all = pd.merge(data1,test1,on='store_id',how='left')
order_all = pd.merge(order_all,test,on='area_cd',how='left')

order_all.loc[order_all['takeout_flag']==0,'takeout_name'] = 'delivery'
order_all.loc[order_all['takeout_flag']==1,'takeout_name'] = 'takeout'

order_all.loc[order_all['status']==0,'status_name'] = '주문 접수'
order_all.loc[order_all['status']==1,'status_name'] = '결제 완료'
order_all.loc[order_all['status']==2,'status_name'] = '배달 완료'
order_all.loc[order_all['status']==9,'status_name'] = '주문 취소'

order_all.loc[:,'order_date'] = pd.to_datetime(
    order_all['order_accept_date']).dt.date

"""
wb = openpyxl.load_workbook('test.xlsx',read_only=True)
ws = wb['Sheet']
print(ws.cell(1,1).value)
wb.close()
"""
store_id = 1
store_df = order_all.loc[order_all['store_id']==store_id].copy()
store_name = store_df['store_name'].unique()[0]
store_sales_total = store_df.loc[
    store_df['status'].isin([1,2])]['total_amount'].sum()
store_sales_takeout = store_df.loc[
    store_df['status']==1]['total_amount'].sum()
store_sales_delivery = store_df.loc[
    store_df['status']==2]['total_amount'].sum()
print(f'매출액 확인 {store_sales_total} = '
      f'{store_sales_takeout + store_sales_delivery}')
output_df = store_df[['order_accept_date','customer_id','total_amount',
                      'takeout_name','status_name']]
output_df.head()

매출액 확인 31253890 = 31253890


,order_accept_date,customer_id,total_amount,takeout_name,status_name
20,2021-05-01 11:01,C88417120,33190,delivery,배달 완료
320,2021-05-01 11:29,C69901556,31900,delivery,배달 완료
330,2021-05-01 11:30,C38622100,26470,delivery,배달 완료
558,2021-05-01 11:49,C44365345,51000,delivery,배달 완료
870,2021-05-01 12:13,C52002978,18820,delivery,주문 취소
